In [10]:
# Feature Extraction from STFT Data

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import entropy

In [11]:
# Paths to STFT data and features output
data_path = r'C:\Users\Sami\Desktop\Uni\Flow-EEG-Psych\data\stft'
features_output_path = r'C:\Users\Sami\Desktop\Uni\Flow-EEG-Psych\data\features'

# Ensure the features directory exists
if not os.path.exists(features_output_path):
    os.makedirs(features_output_path)


In [12]:
# Frequency band definitions (in Hz)
frequency_bands = {
    'delta': (1, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'gamma': (30, 50)
}

# Sampling rate assumption for STFT analysis (modify if different)
sampling_rate = 256

In [13]:
# Function to calculate band power
def calculate_band_power(stft_matrix, freqs, band):
    """
    Calculate the average power in the given frequency band.
    """
    band_freqs = np.where((freqs >= band[0]) & (freqs <= band[1]))[0]
    band_power = np.mean(np.abs(stft_matrix[band_freqs, :] ** 2), axis=0)
    return np.mean(band_power)

# Function to calculate spectral entropy
def calculate_spectral_entropy(stft_matrix):
    """
    Calculate the spectral entropy of the given STFT data.
    """
    power_spectrum = np.abs(stft_matrix) ** 2
    power_spectrum /= np.sum(power_spectrum, axis=0)  # Normalize
    return entropy(power_spectrum, base=2, axis=0).mean()


In [14]:
# Function to extract features from an STFT CSV file
def extract_features_from_file(file_path):
    df = pd.read_csv(file_path)
    # Extract signal values as NumPy array
    stft_data = df.to_numpy()
    
    # Assuming the first column contains frequency information
    frequencies = stft_data[:, 0]
    # The rest are STFT values
    stft_matrix = stft_data[:, 1:]

    # Calculate features for each frequency band
    features = {}
    for band_name, band_range in frequency_bands.items():
        features[f'{band_name}_power'] = calculate_band_power(stft_matrix, frequencies, band_range)
    
    # Calculate spectral entropy
    features['spectral_entropy'] = calculate_spectral_entropy(stft_matrix)

    return features

In [15]:
# Import necessary modules
import os
from src.features.extract_features import main as extract_features_main

# Set the paths to STFT data and features output
stft_data_path = os.path.join(os.getcwd(), 'data', 'stft')
features_output_path = os.path.join(os.getcwd(), 'data', 'features')

# Ensure the features directory exists
if not os.path.exists(features_output_path):
    os.makedirs(features_output_path)

# Run the feature extraction
extract_features_main()



Processing file: S01_B_RWEO_PreOL_stft.csv
Processing file: S01_C_OLoop_stft.csv
Processing file: S01_D_RWEO_PstOL_stft.csv
Processing file: S01_E_RWEO_PreCL_stft.csv
Processing file: S01_F_CL_Sil_50_100_stft.csv
Processing file: S01_G_RWEO_PstCL_stft.csv
Processing file: S02_B_RWEO_PreOL_stft.csv
Processing file: S02_C_OLoop_stft.csv
Processing file: S02_D_RWEO_PstOL_stft.csv
Processing file: S02_E_RWEO_PreCL_stft.csv
Processing file: S02_F_CL_Sil_50_100_stft.csv
Processing file: S02_G_RWEO_PstCL_stft.csv
Processing file: S03_B_RWEO_PreOL_stft.csv
Processing file: S03_C_OLoop_stft.csv
Processing file: S03_D_RWEO_PstOL_stft.csv
Processing file: S03_E_RWEO_PreCL_stft.csv
Processing file: S03_F_CL_Sil_50_100_stft.csv
Processing file: S03_G_RWEO_PstCL_stft.csv
Processing file: S04_B_RWEO_PreOL_stft.csv
Processing file: S04_C_OLoop_stft.csv
Processing file: S04_D_RWEO_PstOL_stft.csv
Processing file: S04_F_CL_Sil_50_100_stft.csv
Processing file: S04_G_RWEO_PstCL_stft.csv
Processing file: S0